In [7]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=dcef76e4af5950a49985ed19aed082368df5ed7a659f62daf2acff00875d8eff
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [8]:
import os
import sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [9]:
from pyspark.sql.window import Window

In [10]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('PySpark_02')
    .getOrCreate()
)

In [11]:
df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/wc2018-players.csv', header=True, inferSchema=True)

In [12]:
df.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
|Argentina|  4|  DF|  ANSALDI Cristian|20.09.1986|   ANSALDI|     Torino FC (ITA)|   181|    73|
|Argentina|  5|  MF|      BIGLIA Lucas|30.01.1986|    BIGLIA|      AC Milan (ITA)|   175|    73|
|Argentina|  7|  MF|       BAN

Aterações DIA01

In [13]:
df = df.withColumnRenamed('Team', 'Selecao').withColumnRenamed('#', 'Numero').withColumnRenamed('Pos.', 'Posicao')\
.withColumnRenamed('FIFA Popular Name', 'Nome_FIFA').withColumnRenamed('Birth Date', 'Nascimento')\
.withColumnRenamed('Shirt Name', 'Nome Camiseta').withColumnRenamed('Club', 'Time').withColumnRenamed('Height', 'Altura')\
.withColumnRenamed('Weight', 'Peso')

In [14]:
dia = udf(lambda data: data.split('.')[0])
mes = udf(lambda data: data.split('.')[1])
ano = udf(lambda data: data.split('.')[2])

In [15]:
df = df.withColumn('Dia', dia('Nascimento')).withColumn('Mes', mes('Nascimento')).withColumn('Ano', ano('nascimento'))
df = df.withColumn('Data_Nascimento', concat_ws('-', 'Ano', 'Mes', 'Dia').cast(DateType()))
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+---+----+---------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+---+----+---------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|     1992-08-31|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|     1996-01-21|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66| 15| 02|1993|     1993-02-15|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|     1990-07-13|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI

In [16]:
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Dia: string (nullable = true)
 |-- Mes: string (nullable = true)
 |-- Ano: string (nullable = true)
 |-- Data_Nascimento: date (nullable = true)



Fim das Alterações

Drop Colulas

In [17]:
df = df.drop('Nascimento')

BackUp

In [ ]:
df2 = df

#### Window Ranking Functions

- Window Function 1 - Numero de linhas - row_number()
- Window Function 2 - Ranking 1 - rank()
- Window Function 3 - Ranking 2 - dense_rank()
- Window Function 4 - Porcentagem Ranking - percent_rank()
- Window Function 5 - Divisão em ' N ' partes - ntile()

Window Function 1 - Numero de linhas - row_number()

In [ ]:
num_linhas = Window.partitionBy('Selecao').orderBy(desc('Altura'))

df.withColumn('numero_linha', row_number().over(num_linhas)).show(50)

Window Function 2 - Ranking 1 - rank()

In [ ]:
rank1 = Window.partitionBy('selecao').orderBy(desc('altura'))

df.withColumn('rank', rank().over(rank1)).show(50)

Window Function 3 - Ranking 2 - rank()

In [ ]:
rank2 = Window.partitionBy('selecao').orderBy(desc('altura'))
df.withColumn('rank2', dense_rank().over(rank2)).show(50)

Window Function 4 - Porcentagem Ranking - percent_rank()

In [ ]:
porcentagem = Window.partitionBy('selecao').orderBy(desc('Altura'))
df.withColumn('%', percent_rank().over(porcentagem)).show(50)

Window Function 5 - Divisão em 'n' partes - ntile()

In [30]:
divisao = Window.partitionBy('selecao').orderBy(desc('Altura'))
df.withColumn('div', ntile(5).over(divisao)).show(50)

+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+---+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|div|
+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+---+
|Argentina|     6|     DF|    FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|  1|
|Argentina|     1|     GK|     GUZMAN Nahuel|       GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1986-02-10|  1|
|Argentina|    16|     DF|       ROJO Marcos|         ROJO|Manchester United...|   189|  82| 20| 03|1990|     1990-03-20|  1|
|Argentina|    12|     GK|     ARMANI Franco|       ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|     1986-10-16|  1|
|Argentina|    23|     GK|CABALLERO Wilfredo|    CABALLERO|    Chelsea FC (ENG)|   186|  80| 28| 09|1981|     1981-09-

In [33]:
df = df.drop('nome_fifa')

Window Anallytic Functions (funções Analiticas)

Window Function 6 - LAG e Lead

In [ ]:
degrau = Window.partitionBy('selecao').orderBy(desc('Altura'))
df.withColumn('degrau', lag('Peso').over(degrau)).show(50)

In [ ]:
degrau = Window.partitionBy('selecao').orderBy(desc('Altura'))
df.withColumn('degrau', lead('Peso').over(degrau)).show(50)

Agragações

GroupBy + AGG 1

In [39]:
df.groupBy('selecao').agg({'altura':'avg'}).orderBy('avg(altura)', ascending=False).show()

+--------------+------------------+
|       selecao|       avg(altura)|
+--------------+------------------+
|        Serbia|186.69565217391303|
|       Denmark| 186.6086956521739|
|       Germany| 185.7826086956522|
|        Sweden| 185.7391304347826|
|       Iceland|185.52173913043478|
|       Belgium|185.34782608695653|
|       Croatia| 185.2608695652174|
|       Nigeria|184.52173913043478|
|       IR Iran|184.47826086956522|
|        Russia| 184.3913043478261|
|       Senegal|183.65217391304347|
|        France|183.30434782608697|
|        Poland|183.17391304347825|
|       Tunisia|183.08695652173913|
|   Switzerland|182.91304347826087|
|       England| 182.7391304347826|
|       Morocco|182.69565217391303|
|        Panama|182.17391304347825|
|Korea Republic| 181.8695652173913|
|       Uruguay|181.04347826086956|
+--------------+------------------+
only showing top 20 rows



Agg 2

In [40]:
df.groupBy('selecao').agg(avg('altura')).orderBy('avg(altura)').show()

+--------------+------------------+
|       selecao|       avg(altura)|
+--------------+------------------+
|          Peru| 177.6086956521739|
|  Saudi Arabia|177.65217391304347|
|     Argentina|178.43478260869566|
|         Japan| 178.7826086956522|
|      Portugal| 179.7391304347826|
|        Mexico| 179.7826086956522|
|         Spain|179.91304347826087|
|    Costa Rica|180.69565217391303|
|        Brazil| 180.7826086956522|
|      Colombia| 180.7826086956522|
|     Australia| 180.8695652173913|
|         Egypt|             181.0|
|       Uruguay|181.04347826086956|
|Korea Republic| 181.8695652173913|
|        Panama|182.17391304347825|
|       Morocco|182.69565217391303|
|       England| 182.7391304347826|
|   Switzerland|182.91304347826087|
|       Tunisia|183.08695652173913|
|        Poland|183.17391304347825|
+--------------+------------------+
only showing top 20 rows



Where

In [ ]:
df.where('selecao = "Brazil"').show()

In [47]:
top1 = Window.partitionBy('selecao').orderBy(desc('altura'))

df.withColumn('Top', rank().over(top1)).where('top = "1"').show(50)

+--------------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+---+
|       Selecao|Numero|Posicao|Nome Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|Top|
+--------------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+---+
|     Argentina|     6|     DF|        FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|  1|
|     Australia|    12|     GK|        JONES|Feyenoord Rotterd...|   193|  87| 19| 03|1982|     1982-03-19|  1|
|       Belgium|     1|     GK|     COURTOIS|    Chelsea FC (ENG)|   199|  91| 11| 05|1992|     1992-05-11|  1|
|        Brazil|    16|     GK|       CASSIO|SC Corinthians (BRA)|   195|  92| 06| 06|1987|     1987-06-06|  1|
|      Colombia|    13|     DF|      Y. MINA|  FC Barcelona (ESP)|   194|  95| 23| 09|1994|     1994-09-23|  1|
|    Costa Rica|    19|     DF|    K. WASTON|Vancouver Whiteca...|   196|  87| 01| 01|1988|     1988-01-

Describe

In [48]:
df.describe().show()

+-------+---------+-----------------+-------+-------------+--------------------+-----------------+-----------------+------------------+------------------+------------------+
|summary|  Selecao|           Numero|Posicao|Nome Camiseta|                Time|           Altura|             Peso|               Dia|               Mes|               Ano|
+-------+---------+-----------------+-------+-------------+--------------------+-----------------+-----------------+------------------+------------------+------------------+
|  count|      736|              736|    736|          736|                 736|              736|              736|               736|               736|               736|
|   mean|     NULL|             12.0|   NULL|         NULL|                NULL|182.4076086956522|77.18885869565217|15.793478260869565|5.8790760869565215| 1990.110054347826|
| stddev|     NULL|6.637760461599851|   NULL|         NULL|                NULL|6.930924233929302|7.233778346883639| 8.76112382873

In [49]:
df.filter('selecao ="Brazil"').describe().show()


+-------+-------+-----------------+-------+-------------+--------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|Selecao|           Numero|Posicao|Nome Camiseta|                Time|           Altura|             Peso|              Dia|              Mes|              Ano|
+-------+-------+-----------------+-------+-------------+--------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|     23|               23|     23|           23|                  23|               23|               23|               23|               23|               23|
|   mean|   NULL|             12.0|   NULL|         NULL|                NULL|180.7826086956522|76.56521739130434|11.26086956521739|6.130434782608695|1989.391304347826|
| stddev|   NULL|6.782329983125267|   NULL|         NULL|                NULL|7.354383490255254|8.239737898283606|6.876953549252538|2.784769418006175|3.499

Window Function 8 - Função de agregação usando Window Function

In [56]:
parametro = Window.partitionBy('selecao').orderBy(desc('altura'))
parametro2 = Window.partitionBy('selecao')

df.withColumn('linha_x', row_number().over(parametro))\
.withColumn('media', avg('altura').over(parametro2))\
.withColumn('max', max('altura').over(parametro2))\
.withColumn('min', min('altura').over(parametro2))\
.filter('linha_x = "1"').select('selecao', 'Media','Max','Min')\
.orderBy('media', ascending=False).show(50)

+--------------+------------------+---+---+
|       selecao|             Media|Max|Min|
+--------------+------------------+---+---+
|        Serbia|186.69565217391303|195|169|
|       Denmark| 186.6086956521739|200|171|
|       Germany| 185.7826086956522|195|176|
|        Sweden| 185.7391304347826|198|177|
|       Iceland|185.52173913043478|198|170|
|       Belgium|185.34782608695653|199|169|
|       Croatia| 185.2608695652174|201|172|
|       Nigeria|184.52173913043478|197|172|
|       IR Iran|184.47826086956522|194|177|
|        Russia| 184.3913043478261|196|173|
|       Senegal|183.65217391304347|196|173|
|        France|183.30434782608697|197|168|
|        Poland|183.17391304347825|195|172|
|       Tunisia|183.08695652173913|192|170|
|   Switzerland|182.91304347826087|192|165|
|       England| 182.7391304347826|196|170|
|       Morocco|182.69565217391303|190|167|
|        Panama|182.17391304347825|197|165|
|Korea Republic| 181.8695652173913|197|170|
|       Uruguay|181.043478260869